In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD

# Initialize ImageDataGenerator for training with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale the images to be in range [0, 1]
    shear_range=0.2,  # Shear transformation for augmentation
    zoom_range=0.2,   # Zoom transformation for augmentation
    horizontal_flip=True,  # Random horizontal flip for augmentation
    rotation_range=40,  # Random rotation for augmentation
    width_shift_range=0.2,  # Random width shift for augmentation
    height_shift_range=0.2,  # Random height shift for augmentation
    validation_split=0.2  # Split data into training (80%) and validation (20%)
)

# Load training images from directory and apply data augmentation
training_set = train_datagen.flow_from_directory(
    r'C:\Users\ADMIN\OneDrive\Desktop\CatsVsDogs',  # Path to the dataset folder
    target_size=(150, 150),     # Resize the images to 150x150
    batch_size=32,              # Set the batch size
    class_mode='categorical',   # Categorical classification (multiple classes)
    subset='training'           # Subset for training data (80% of the total data)
)

# Load validation images from directory (no augmentation applied, just rescaling)
validation_set = train_datagen.flow_from_directory(
    r'C:\Users\ADMIN\OneDrive\Desktop\CatsVsDogs',  # Path to the dataset folder
    target_size=(150, 150),     # Resize the images to 150x150
    batch_size=32,              # Set the batch size
    class_mode='categorical',   # Categorical classification (multiple classes)
    subset='validation'         # Subset for validation data (20% of the total data)
)

# Initialize VGG16 model for feature extraction (exclude the top layers)
base_model = VGG16(
    weights='imagenet',         # Pre-trained weights on ImageNet
    include_top=False,          # Exclude the fully connected layers (top layers)
    input_shape=(150, 150, 3)   # Set the input size (150x150 images)
)

# Freeze the layers of VGG16 to prevent training (feature extraction only)
for layer in base_model.layers:
    layer.trainable = False

# Build the model with custom fully connected layers
model = Sequential()

# Add the pre-trained VGG16 model as the feature extractor
model.add(base_model)

# Add custom fully connected layers
model.add(Flatten())                    # Flatten the output from the convolutional layers
model.add(Dense(128, activation='relu')) # Add a fully connected layer with 128 neurons
model.add(Dense(2, activation='softmax'))  # Output layer for multi-class classification (2 classes)

# Compile the model with stochastic gradient descent (SGD) optimizer and categorical cross-entropy loss
model.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),  # Use SGD with learning rate and momentum
    loss='categorical_crossentropy',  # Categorical cross-entropy for multi-class classification
    metrics=['accuracy']  # Evaluate model based on accuracy
)


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [5]:

# Train the model with training and validation sets
model.fit(
    training_set,
    epochs=15,
    validation_data=validation_set  # Use validation data during training
)

Epoch 1/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 205s 4s/step - accuracy: 0.6172 - loss: 0.6606 - val_accuracy: 0.8025 - val_loss: 0.6087
Epoch 2/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 253s 5s/step - accuracy: 0.7471 - loss: 0.5382 - val_accuracy: 0.7650 - val_loss: 0.4798
Epoch 3/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 235s 5s/step - accuracy: 0.6664 - loss: 0.5878 - val_accuracy: 0.7750 - val_loss: 0.4795
Epoch 4/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 229s 5s/step - accuracy: 0.7312 - loss: 0.5696 - val_accuracy: 0.8000 - val_loss: 0.4716
Epoch 5/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 238s 5s/step - accuracy: 0.7333 - loss: 0.5710 - val_accuracy: 0.8275 - val_loss: 0.4193
Epoch 6/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 236s 5s/step - accuracy: 0.7991 - loss: 0.4603 - val_accuracy: 0.7275 - val_loss: 0.5231
Epoch 7/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 175s 4s/step - accuracy: 0.8308 - loss: 0.4153 - val_accuracy: 0.7375 - val_loss: 0.5041
Epoch 8/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 180s 4s/step - accuracy: 0.7983 - loss: 0.4353 - val_accuracy: 0.8400 - v

In [31]:
from tensorflow.keras.preprocessing import image

image_path = r"C:\Users\ADMIN\OneDrive\Desktop\dog.jpg"  # Replace with the actual path to your image
img = image.load_img(image_path, target_size=(150, 150))  # Resize to match model input size



In [32]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [33]:
import numpy as np

# Convert the image to a NumPy array
img_array = image.img_to_array(img)

    # Add a batch dimension (convert from shape (150, 150, 3) to (1, 150, 150, 3))
img_array = np.expand_dims(img_array, axis=0)

    # Rescale the pixel values to [0, 1]
img_array = img_array / 255.0
prediction = model.predict(img_array)
confidence = np.max(prediction)
predicted_label_index = np.argmax(prediction)
print(f"Predicted Label Index: {predicted_label_index}")
print(f"Confidence: {confidence * 100:.2f}%")
if predicted_label_index == 0:
    print('cats')
else:
    print('dogs')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
Predicted Label Index: 1
Confidence: 91.32%
dogs
